In [ ]:
import math



def read_data_from_file(filename):
    blocks = {}
    route_points = []
    order = []
    current_block = None

    with open(filename, 'r') as file:
        for line in file:
            line = line.split('#')[0].strip()  
            if not line:
                continue

            if line == "DATA":
                current_block = "DATA"
            elif line == "ROUTE":
                current_block = "ROUTE"
            elif line == "ORDER":
                current_block = "ORDER"
            elif line == "/":
                current_block = None
            else:
                if current_block == "DATA":
                    parts = line.split()
                    if len(parts) == 3:
                        block_type = parts[0]
                        count = int(parts[1])
                        cost = int(parts[2])
                        if block_type.startswith("L"):
                            length = int(block_type[1:])
                            blocks[block_type] = {
                                "type": "linear",
                                "length": length,
                                "count": count,
                                "cost": cost,
                            }
                        elif block_type.startswith("T"):
                            angle = math.pi/int(block_type[1:]) 
                            radius = 3  
                            blocks[block_type] = {
                                "type": "turn",
                                "angle": angle,
                                "radius": radius,
                                "count": count,
                                "cost": cost,
                            }
                        elif block_type.startswith("B"):
                            length = 4  
                            blocks[block_type] = {
                                "type": "bridge",
                                "length": length,
                                "count": count,
                                "cost": cost,
                            }
                elif current_block == "ROUTE":
                    parts = line.split()
                    if len(parts) == 3:
                        x, y, cost = map(int, parts)
                        route_points.append((x, y, cost))
                elif current_block == "ORDER":
                    parts = line.split()
                    if len(parts) == 2:
                        order.append((parts[0], int(parts[1])))
    return blocks, route_points, order


def check_blocks_usage(blocks, order):
    used_blocks = {key: 0 for key in blocks}
    for block, step in order:
        if block in used_blocks:
            used_blocks[block] += abs(step)
            if used_blocks[block] > blocks[block]["count"]:
                raise ValueError(f"В блоке ORDER блоков {block} используется в больше, чем их есть в DATA")#
        else:
            raise ValueError(f"Неизвестный тип блока в ORDER: {block}")#



def calculate_final_position(blocks, order):
    x, y = 0, 0
    direction = [0, 1]  

    #print("Координаты после каждого блока:")
    #print(f"Начальная позиция: x={x}, y={y}, направление=({direction[0]:.3f}, {direction[1]:.3f})")

    for block, step in order:
        if block in blocks:
            block_data = blocks[block]
            if block_data["type"] == "linear":
                
                length = block_data["length"] * step
                x += direction[0] * length
                y += direction[1] * length
            elif block_data["type"] == "turn":
                
                angle = block_data["angle"] * step
                cos_angle = math.cos(-angle)
                sin_angle = math.sin(-angle)

                
                direction = [
                    direction[0] * cos_angle - direction[1] * sin_angle,
                    direction[0] * sin_angle + direction[1] * cos_angle
                ]
               
                magnitude = math.sqrt(direction[0]**2 + direction[1]**2)
                direction = [direction[0] / magnitude, direction[1] / magnitude]
               
                radius = block_data["radius"]

                dy = radius * (math.sin(angle))
                dx = radius * (1 - math.cos(angle))
                x += -direction[1] * dx + direction[0] * dy
                y += direction[0] * dx + direction[1] * dy
            
            #print(f"После блока {block} с шагом {step}: x={x:.6f}, y={y:.6f}, направление=({direction[0]:.3f}, {direction[1]:.3f})")

    return round(x, 6), round(y, 6)



def check_route_ending(blocks, order):
    final_x, final_y = calculate_final_position(blocks, order)
    if (final_x, final_y) != (0, 0):
        raise ValueError(f"Путь должен заканчиваться в точке (0,0), но исходя из ORDER он заканчивается в ({final_x}, {final_y})")#



def check_route_section(route_points):
    if not route_points:
        raise ValueError("Отсутствует блок ROUTE")#



def process_file(filename):
    blocks, route_points, order = read_data_from_file(filename)

    try:
        check_route_section(route_points)  
        check_blocks_usage(blocks, order) 
        check_route_ending(blocks, order)
        print("Файл успешно обработан, ошибок не обнаружено.")
    except ValueError as e:
        print(f"Ошибка: {e}")


filename = "LILA_8.txt" 


process_file(f"../../tests/{filename}")


Ошибка: Путь должен заканчиваться в точке (0,0), но исходя из ORDER он заканчивается в (8.414214, 0.0)
